# Diagnóstico de calidad de una base de datos
###Contexto:
Detectar problemas de calidad en una base de datos es el primer paso para garantizar decisiones confiables y procesos eficientes. Esta actividad permite aplicar criterios reales sobre un conjunto de datos para identificar errores frecuentes.

###Consigna:
Analiza una base de datos proporcionada por el docente (o simulada) y detecta problemas de calidad según al menos cinco dimensiones (por ejemplo: completitud, unicidad, validez, consistencia y actualidad).

###Paso a paso:
1- Revisa la base de datos entregada.

2- Elige cinco dimensiones de calidad a evaluar.

3- Define cómo vas a medir cada una (por ejemplo: porcentaje de nulos, duplicados, valores fuera de rango, etc.).

4- Detecta los problemas presentes y cuantifica su impacto.

5- Documenta los hallazgos en un informe claro y estructurado.

In [30]:
from google.colab import files
import json

# Paso 1: Subir el archivo
uploaded = files.upload()

# Paso 2: Leer el archivo JSON (reemplaza 'nombre_del_archivo.json' por el nombre real)
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Paso 3: Ver los primeros elementos
print(data[:3])  # Si es una lista de objetos


Saving tickets_soporte.json to tickets_soporte (1).json
[{'ticket_id': 'TKT00001', 'cliente_id': 'CL003', 'descripcion': 'Positive while rate statement position someone popular theory reveal choose relate raise tonight factor.', 'fecha_apertura': '2025-02-10T02:12:20', 'fecha_cierre': '2025-02-11T04:12:20', 'estado': 'Cerrado', 'agente': 'Laura Moon', 'prioridad': 'Media', 'canal': 'Correo', 'evaluacion_cliente': 2}, {'ticket_id': 'TKT00001', 'cliente_id': 'CL011', 'tema': 'Consulta técnica', 'descripcion': 'Meet special leader those know figure year real hear than exactly group painting many record friend.', 'fecha_apertura': '2026-07-30T22:17:28.457786', 'fecha_cierre': '2024-12-30T09:52:00', 'estado': 'Cerrado', 'agente': 'Jason Wright', 'prioridad': 'Baja', 'canal': 'Formulario Web', 'evaluacion_cliente': 3}, {'ticket_id': 'TKT00003', 'cliente_id': 'CL011', 'tema': 'Problemas de acceso', 'descripcion': 'Pay grow threat fine pressure improve media learn thank.', 'fecha_apertura': 'f

In [31]:
import pandas as pd

df = pd.DataFrame(data)
df.head()


,ticket_id,cliente_id,descripcion,fecha_apertura,fecha_cierre,estado,agente,prioridad,canal,evaluacion_cliente,tema
0,TKT00001,CL003,Positive while rate statement position someone...,2025-02-10T02:12:20,2025-02-11T04:12:20,Cerrado,Laura Moon,Media,Correo,2.0,NaN
1,TKT00001,CL011,Meet special leader those know figure year rea...,2026-07-30T22:17:28.457786,2024-12-30T09:52:00,Cerrado,Jason Wright,Baja,Formulario Web,3.0,Consulta técnica
2,TKT00003,CL011,Pay grow threat fine pressure improve media le...,fecha inválida,None,En Proceso,Craig Ward,Baja,Correo,NaN,Problemas de acceso
3,TKT00004,CL013,Crime teach identify test Mrs dog strategy ord...,2024-10-15T23:44:54,None,Cerrado,Jason Wright,Baja,Correo,2.0,Error en factura
4,TKT00005,CL007,Should network compare often radio eat local T...,2026-07-30T22:14:07.386811,None,En Proceso,Douglas King,Media,Correo,NaN,Solicitud de reembolso


In [32]:
# Porcentaje de valores nulos por columna
completitud = df.isnull().mean() * 100
print("🧱 Completitud (% de valores nulos por columna):")
print(completitud)
########################################################

# Revisar duplicados en los IDs de ticket
duplicados = df.duplicated(subset='ticket_id').sum()
print(f"🔁 Número de tickets duplicados: {duplicados}")
########################################################
# Validación de fechas
def validar_fecha(fecha):
    try:
        pd.to_datetime(fecha)
        return True
    except:
        return False

df['fecha_apertura_valida'] = df['fecha_apertura'].apply(validar_fecha)
df['fecha_cierre_valida'] = df['fecha_cierre'].apply(lambda x: validar_fecha(x) if pd.notnull(x) else True)

fechas_invalidas = (~df['fecha_apertura_valida'] | ~df['fecha_cierre_valida']).sum()
print(f"📅 Registros con fechas inválidas: {fechas_invalidas}")

##########################################################
# Convertimos fechas válidas
df['fecha_apertura_dt'] = pd.to_datetime(df['fecha_apertura'], errors='coerce')
df['fecha_cierre_dt'] = pd.to_datetime(df['fecha_cierre'], errors='coerce')

# Inconsistencia: fecha de apertura mayor a la de cierre
# Convertir 'fecha_apertura' a datetime, forzando errores como NaT
df['fecha_apertura'] = pd.to_datetime(df['fecha_apertura'], errors='coerce')
inconsistentes = df[(df['fecha_apertura_dt'].notnull()) &
                    (df['fecha_cierre_dt'].notnull()) &
                    (df['fecha_apertura_dt'] > df['fecha_cierre_dt'])]
print(f"⚠️ Registros con fechas inconsistentes: {len(inconsistentes)}")
##########################################################

# Convertimos fechas válidas
df['fecha_apertura_dt'] = pd.to_datetime(df['fecha_apertura'], errors='coerce')
df['fecha_cierre_dt'] = pd.to_datetime(df['fecha_cierre'], errors='coerce')

# Inconsistencia: fecha de apertura mayor a la de cierre
inconsistentes = df[(df['fecha_apertura_dt'].notnull()) &
                    (df['fecha_cierre_dt'].notnull()) &
                    (df['fecha_apertura_dt'] > df['fecha_cierre_dt'])]
print(f"⚠️ Registros con fechas inconsistentes: {len(inconsistentes)}")

####################################################
from datetime import datetime

hoy = pd.to_datetime(datetime.today())
futuro = df[df['fecha_apertura_dt'] > hoy]
print(f"🕒 Registros con fecha de apertura en el futuro: {len(futuro)}")



🧱 Completitud (% de valores nulos por columna):
ticket_id              0.0
cliente_id             0.0
descripcion            3.0
fecha_apertura         0.0
fecha_cierre          54.9
estado                 3.2
agente                 0.0
prioridad              0.0
canal                  0.0
evaluacion_cliente    50.9
tema                   4.5
dtype: float64
🔁 Número de tickets duplicados: 73
📅 Registros con fechas inválidas: 100
⚠️ Registros con fechas inconsistentes: 0
⚠️ Registros con fechas inconsistentes: 0
🕒 Registros con fecha de apertura en el futuro: 0


In [33]:
import pandas as pd

df = pd.DataFrame(data)
df.head(10)


,ticket_id,cliente_id,descripcion,fecha_apertura,fecha_cierre,estado,agente,prioridad,canal,evaluacion_cliente,tema
0,TKT00001,CL003,Positive while rate statement position someone...,2025-02-10T02:12:20,2025-02-11T04:12:20,Cerrado,Laura Moon,Media,Correo,2.0,NaN
1,TKT00001,CL011,Meet special leader those know figure year rea...,2026-07-30T22:17:28.457786,2024-12-30T09:52:00,Cerrado,Jason Wright,Baja,Formulario Web,3.0,Consulta técnica
2,TKT00003,CL011,Pay grow threat fine pressure improve media le...,fecha inválida,None,En Proceso,Craig Ward,Baja,Correo,NaN,Problemas de acceso
3,TKT00004,CL013,Crime teach identify test Mrs dog strategy ord...,2024-10-15T23:44:54,None,Cerrado,Jason Wright,Baja,Correo,2.0,Error en factura
4,TKT00005,CL007,Should network compare often radio eat local T...,2026-07-30T22:14:07.386811,None,En Proceso,Douglas King,Media,Correo,NaN,Solicitud de reembolso
5,TKT00006,CL007,Travel effect old three hear arrive oil easy s...,2026-07-30T22:17:28.457756,None,En Proceso,Dr. Elizabeth Conner,Baja,Teléfono,NaN,NaN
6,TKT00001,CL003,Billion measure ability which today them maybe...,2026-07-30T22:17:28.458007,None,Abierto,Joan Friedman,Media,Formulario Web,NaN,Solicitud de reembolso
7,TKT00008,CL002,Agreement lot feeling shoulder cold rate go se...,fecha inválida,2025-03-21T12:53:01,Cerrado,Dr. Elizabeth Conner,Media,Chat,5.0,Solicitud de reembolso
8,TKT00009,CL014,Lead term report join follow democratic scienc...,2025-04-16T02:32:38,None,Cerrado,Dr. Elizabeth Conner,Alta,Correo,NaN,Cambio de plan
9,TKT00010,CL001,Hand the term tough discussion candidate campa...,2026-07-30T22:14:07.386835,2024-09-20T12:44:48,Cerrado,Brittany Wilson,Alta,Formulario Web,5.0,Problemas con la app


#Diagnóstico de Calidad de Datos para tickets_soporte.json
###1. Completitud
La mayoría de columnas importantes como ticket_id, cliente_id, agente, prioridad y canal no tienen valores nulos (0%).

Columnas con alta incompletitud:

fecha_cierre: 54.9% de valores nulos. Esto puede deberse a tickets abiertos o sin cerrar.

evaluacion_cliente: 50.9% de valores nulos, limitando análisis de satisfacción.

descripcion: 3% nulos.

estado: 3.2% nulos.

tema: 4.5% nulos.

Impacto:
La falta de fechas de cierre y evaluaciones limita análisis de ciclo de vida y satisfacción. La incompletitud en tema y descripcion afecta categorización y contexto.

###2. Unicidad
Se detectaron 73 tickets con ticket_id duplicado.

Ejemplo: TKT00001 aparece varias veces.

Impacto:
La duplicidad de IDs compromete la integridad de la base, provoca errores en conteo y seguimiento.

###3. Validez
Se detectaron 100 registros con fechas inválidas (como "fecha inválida" en fecha_apertura).

Esto afecta el análisis temporal y filtrado por fechas.

Impacto:
Datos con formato erróneo no pueden ser procesados correctamente en análisis de tiempo o métricas temporales.

###4. Consistencia
No se detectaron registros con fecha_apertura posterior a fecha_cierre (0 registros inconsistentes).

Sin embargo, hay registros con estado "Cerrado" y sin fecha de cierre (ver ejemplo).

Impacto:
Inconsistencias en estado y fechas afectan confiabilidad del ciclo de vida del ticket.

###5. Actualidad
No hay registros con fecha_apertura en el futuro (0 registros).

Sin embargo, fechas en años como 2026 pueden ser sospechosas si el dataset es actual.

Impacto:
Datos que parecen del futuro pueden indicar errores de carga o configuración incorrecta.

##Propuesta de Mejora para la Calidad de Datos
1-Limpieza de fechas inválidas
Reemplazar textos no válidos en fechas por valores nulos para un correcto análisis.

2-Manejo de valores nulos
Evaluar y completar datos faltantes en campos clave como fecha_cierre, evaluación y descripción.

3-Duplicados
Identificar y eliminar o corregir registros con ticket_id repetidos para mantener la integridad.

4-Validación de consistencia
Asegurar coherencia entre fechas y estado (por ejemplo, tickets cerrados con fecha de cierre).

5-Revisión de fechas futuras
Corregir fechas fuera de rango o futuras que no correspondan a la realidad.